### Steps before using this function

In [0]:
'''
1. Go to https://portal.azure.com/#view/Microsoft_AAD_RegisteredApps/ApplicationsListBlade and create a new application (Give it a name and register). You have created your Service Principal (SP).
2. Under Manage on the left Side Bar click on API permissions and add the following APPLICATION permissions for Microsoft Graph:

                        - Sites.Selected 

3. Once you have added the permission you will need to send Hal Sclater the following information:

                    - Application Name
                    - Site URL for the sites you want access to

4. Click on Certificates & Secrets and then New Client Secret, add a description (e.g. "Python API Access") and select a duration that suits your scenario. Copy the generated secret and 
   save it in a safe location. You will not be able to retrieve it again.
5. Create a secret scope either linked to a KeyVault or Databricks Secret Scope.
6. Add the Generated Secret from step 4 to the scope.
7. Give a Read permission to the secret scope for the Service Principle that will be running the notebook.
8. Give the SP permissions to the directories you want to access. 



'''

In [0]:
%python
def Downlaod_Files_From_SharePoint(
    tenant_id: str,
    client_id: str,
    scope: str,
    key: str,
    out_dir: str,
    site_name: str,
    document_library: str,
    documents = 'All', # default as all documents else give a list of documents that are requrired to be downloaded
    version = 'False'
    ):

    import requests
    import json
    import os


    ## Get auth token
     
    url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token'

    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': dbutils.secrets.get(scope=scope, key=key),
        'scope': 'https://graph.microsoft.com/.default'
    }

    response = requests.post(url, headers=headers, data=data)
    access_token = response.json().get('access_token')

    ## Get Site ID

    site_url = f'ukpowernetworks.sharepoint.com:/sites/{site_name}'

    
    url = f'https://graph.microsoft.com/v1.0/sites/{site_url}?$select=id'



    headers = {
        'Authorization': f'Bearer {access_token}',
        'Accept': 'application/json'
    }

    # Send the GET request
    response = requests.get(url, headers=headers)

    site_id = response.json()['id']

    

    ## Get Document Library ID

    

    url = f'https://graph.microsoft.com/v1.0/sites/{site_id}/drives'



    headers = {
        'Authorization': f'Bearer {access_token}',
        'Accept': 'application/json'
    }

    # Send the GET request
    response = requests.get(url, headers=headers)

    # print(response.json()['value'][0]['name'])

    for item in response.json()['value']:
        if item['name'] == document_library:
            library_id = item['id']
            break
        else:
            print('Drive Not Found')


    def download_document(drive_ID, item_ID, file_name, output_path):
        download_endpoint = f'https://graph.microsoft.com/v1.0/drives/{drive_ID}/items/{item_ID}/content'
        response = requests.get(download_endpoint, headers=headers)
    
        if response.status_code == 200:
            full_output_path = os.path.join(output_path, file_name)
            with open(full_output_path, 'wb') as file:
                file.write(response.content)
                print(f'Successfully downloaded {file_name} to {output_path}')
        else:
            print(f'Error downloading {file_name}: {response.status_code} - {response.text}')


    

    endpoint = f'https://graph.microsoft.com/v1.0/sites/{site_id}/drives/{library_id}/root/children'


    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(endpoint, headers=headers)
    items_data = response.json()


    if documents == 'All':
        document_ids = [item['id'] for item in items_data['value'][:2]]
        print(f'Document IDs: {document_ids}')
        out_dir = out_dir

        if not os.path.exists(out_dir):
            os.makedirs(out_dir)

        for item in items_data['value']:
            document_id = item['id']
            document_name = item['name']
            download_document(library_id, document_id, document_name, out_dir)
    else:
        for item in items_data['value']:
            if item['name'] in documents:
                document_id = item['id']
                document_name = item['name']
                download_document(library_id, document_id, document_name, out_dir)

In [0]:
def Downlaod_Files_From_SharePoint(
    tenant_id: str,
    client_id: str,
    scope: str,
    key: str,
    out_dir: str,
    site_name: str,
    document_library: str,
    documents='All'
):
    import requests
    import os
    import pandas as pd
    import shutil
    import datetime

    base_api_url = 'https://graph.microsoft.com/v1.0'
    # Auth
    token_url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token'
    token_data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': dbutils.secrets.get(scope=scope, key=key),
        'scope': 'https://graph.microsoft.com/.default'
    }
    access_token = requests.post(token_url, data=token_data).json().get('access_token')
    headers = {'Authorization': f'Bearer {access_token}'}

    ## Get Site ID
    site_resp = requests.get(
        f'{base_api_url}/sites/ukpowernetworks.sharepoint.com:/sites/{site_name}?$select=id',
        headers=headers)
    site_id = site_resp.json()['id']

    ## Get Document Library ID
    drives_resp = requests.get(f'{base_api_url}/sites/{site_id}/drives', headers=headers)
    library_id = next((d['id'] for d in drives_resp.json()['value'] if d['name'] == document_library), None)
    if not library_id:
        print("Document library not found.")
        return

    # Download function
    def download_file(drive_id, item_id, save_path):
        url = f'{base_api_url}/drives/{drive_id}/items/{item_id}/content'
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            with open(save_path, 'wb') as f:
                f.write(response.content)
                print(f"Downloaded: {save_path}")
        else:
            print(f"Failed to download {save_path} - {response.status_code}")

    ## Look through all files and add all/chosen files to a dictionary
    file_index = {}

    def recurse_items(parent_id, current_path=''):
        endpoint = f'{base_api_url}/drives/{library_id}/items/{parent_id}/children'
        resp = requests.get(endpoint, headers=headers)
        if resp.status_code != 200:
            return
        for item in resp.json()['value']:
            name = item['name']
            item_id = item['id']
            if 'folder' in item:
                recurse_items(item_id, os.path.join(current_path, name))
            else:
                full_path = os.path.join(current_path, name)
                file_index[full_path] = item_id

    # Start recursion from root
    root_id = requests.get(f'{base_api_url}/drives/{library_id}/root', headers=headers).json()['id']
    recurse_items(root_id)

    def normalize_df(df):
        return df.sort_index(axis=1).sort_values(by=df.columns.tolist()).reset_index(drop=True)

    ROOT_DIR = '/Workspace/Users/eri.krasniqi@ukpowernetworks.co.uk/excel_versioning'
    


    def file_comparison(file1, file2):
        if file1.endswith('.xlsx') and file2.endswith('.xlsx'):
            current_file = pd.read_excel(file1)
            new_file = pd.read_excel(file2)
            if not current_file.equals(new_file):
                versions_dir = os.path.join(ROOT_DIR, 'versions')
                os.makedirs(versions_dir, exist_ok=True)
                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                filename = os.path.basename(file1)

                versioned_name = f"{os.path.splitext(filename)[0]}_{timestamp}.xlsx"
                versioned_path = os.path.join(versions_dir, versioned_name)

                shutil.move(file1, versioned_path)
                



    ## Download Section
    if documents == 'All':
        for path, item_id in file_index.items():
            download_file(library_id, item_id, os.path.join(out_dir, path))
    else:
        for doc in documents:
            if doc in file_index:
                download_file(library_id, file_index[doc], os.path.join(out_dir, doc))
            else:
                print(f"File not found in SharePoint: {doc}")


# Apply Versioning to function

Versioning_enabled allows for detected changed between downloaded and stored files to be versioned into a folder ('versions').

As of now the only file types which can be versioned are excel files and txt files.

In [0]:
%pip install openpyxl
import openpyxl
import requests
import os
import pandas as pd
import shutil
import datetime
import filecmp

def download_files_from_sharepoint(
    tenant_id: str, ## tenant Id of the app
    client_id: str, ## client Id of the app
    scope: str,  ## scope name
    key: str, ## key name
    out_dir: str,  ## or os.getcwd() if you want the files to be downloaded in the current directory
    site_name: str, ## name of SharePoint Site, spaces removed
    document_library: str,  ## name of Document Library of interest in SharePoint site
    documents='All',  ## default as 'all' documents, else give a list of documents that are requrired to be downloaded
    versioning_enabled=True,  ## If true any changes in the file will be saved in a versions folder
    structured=True, ## Keep strcuture seen in SharePoint
    vers_dest=os.getcwd() ## Destination for the versions folder of versioning is enabled
):
    base_api_url = 'https://graph.microsoft.com/v1.0'

    # Auth
    token_url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token'
    token_data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': dbutils.secrets.get(scope=scope, key=key),
        'scope': 'https://graph.microsoft.com/.default'
    }
    access_token = requests.post(token_url, data=token_data).json().get('access_token')
    headers = {'Authorization': f'Bearer {access_token}'}

    # Get Site ID
    site_resp = requests.get(
        f'{base_api_url}/sites/ukpowernetworks.sharepoint.com:/sites/{site_name}?$select=id',
        headers=headers)
    site_id = site_resp.json()['id']

    # Get Document Library ID
    drives_resp = requests.get(f'{base_api_url}/sites/{site_id}/drives', headers=headers)
    library_id = next((d['id'] for d in drives_resp.json()['value'] if d['name'] == document_library), None)
    if not library_id:
        print("Document library not found.")
        return

    def download_file(drive_id, item_id):
        url = f'{base_api_url}/drives/{drive_id}/items/{item_id}/content'
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.content
        else:
            print(f"Failed to download file - {response.status_code}")
            return None

    # Look through all files and add all/chosen files to a dictionary
    file_index = {}

    def recurse_items(parent_id, current_path=''):
        endpoint = f'{base_api_url}/drives/{library_id}/items/{parent_id}/children'
        resp = requests.get(endpoint, headers=headers)
        if resp.status_code != 200:
            ## print failed to return children and what the status code is as well as explaination
            return
        for item in resp.json()['value']:
            name = item['name']
            item_id = item['id']
            if structured == True:
                if 'folder' in item:
                    recurse_items(item_id, os.path.join(current_path, name))
                else:
                    full_path = os.path.join(current_path, name)
                    file_index[full_path] = item_id
            else:
                if 'folder' in item:
                    recurse_items(item_id,name)
                else:
                    full_path = name
                    file_index[full_path] = item_id

    root_id = requests.get(f'{base_api_url}/drives/{library_id}/root', headers=headers).json()['id']
    recurse_items(root_id)


    def version(target_file):
        versions_dir = os.path.join(vers_dest, 'versions')
        os.makedirs(versions_dir, exist_ok=True)

        timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = os.path.basename(target_file)

        versioned_name = f"{os.path.splitext(filename)[0]}_{timestamp}.xlsx"


        ###############################################################
        ## cap the num of versions by date/count (add param to func) ##
        ###############################################################



        versioned_path = os.path.join(versions_dir, versioned_name)

        # Move the old file to versions folder
        shutil.move(target_file, versioned_path)
        print(f"Moved {target_file} to {versioned_path}")

    # File comparison function
    # for excel file comparison they are converted to tables and then compared
    # comparison is made prior to writing of file in order not to overwrite the file you are comparing
    def file_comparison(downloaded_content, target_file):
        if target_file.endswith('.xlsx'):
            try:
                current_file = pd.read_excel(target_file)
                new_file = pd.read_excel(downloaded_content)
                
                if not current_file.equals(new_file):
                    if versioning_enabled:
                        version(target_file)
                    return True 
                else:
                    print(f"File {target_file} is unchanged.")
                    return False

            except Exception as e:
                print(f"Error comparing files: {e}")
                return False
        elif target_file.endswith('.txt'):
            # print(f"Opening target_file: {target_file}")
            # print(f"Opening downloaded_content: {downloaded_content}")
            # content is being
            with open(target_file, 'r') as current_file:
                if current_file.read() != downloaded_content.decode('utf-8'):
                    if versioning_enabled:
                        version(target_file)
                    return True  
                else:
                    print(f"File {target_file} is unchanged.")
                    return False
        else:
            print(f"Skipping file {target_file} as it is not an Excel or text file.")
            return False

    # Function that downloads all/chosen files
    if documents == 'All':
        for path, item_id in file_index.items():
            downloaded_content = download_file(library_id, item_id)
            if downloaded_content:
                target_file = os.path.join(out_dir, path)
                print(target_file)
                print(f"Downloading: {target_file}")
                if os.path.exists(target_file):
                    if file_comparison(downloaded_content, target_file):
                        os.makedirs(os.path.dirname(target_file), exist_ok=True)
                        with open(target_file, 'wb') as f:
                            f.write(downloaded_content)
                            print(f"Downloaded and saved: {target_file}")
                else:
                    os.makedirs(os.path.dirname(target_file), exist_ok=True)
                    with open(target_file, 'wb') as f:
                        f.write(downloaded_content)
                        print(f"Downloaded and saved: {target_file}")
    else:
        for doc in documents:
            if doc in file_index:
                downloaded_content = download_file(library_id, file_index[doc])
                if downloaded_content:
                    save_path = os.path.join(out_dir, doc)
                    if os.path.exists(save_path):
                        if file_comparison(downloaded_content, save_path):
                            os.makedirs(os.path.dirname(save_path), exist_ok=True)
                            with open(save_path, 'wb') as f:
                                f.write(downloaded_content)
                                print(f"Downloaded and saved: {save_path}")
                    else:
                        os.makedirs(os.path.dirname(save_path), exist_ok=True)
                        with open(save_path, 'wb') as f:
                            f.write(downloaded_content)
                            print(f"Downloaded and saved: {save_path}")
            else:
                print(f"File not found in SharePoint: {doc}")


In [0]:

download_files_from_sharepoint(
    tenant_id='887a239c-e092-45fe-92c8-d902c3681567',  
    client_id='622b5c85-96fe-4c12-95fe-2cfced5f0f73',  
    scope="sharePoint_conn_tax_proj", 
    key="Sharepoint_graph_api_key",  
    out_dir='Folder1', 
    site_name='GenAiProjectSandbox', 
    document_library='00_Guidelines', 
    documents=['investment_drivers.xlsx'], 
    versioning_enabled=True,  
    structured=True,  
    vers_dest='Folder1' 
)

In [0]:
import pandas as pd

# Load the Excel file into a DataFrame
file_path = 'Folder2/investment_drivers.xlsx'
old_vers_path = 'versions/investment_drivers_20250422_134829.xlsx'
df = pd.read_excel(old_vers_path, dtype={'rule_name': str})

# Display the DataFrame
display(df)